In [7]:
import pandas as pd
import sqlite3
import json
import os
from pprint import pprint
#from dotenv import load_dotenv
import os

In [8]:
#load_dotenv()
PATH_TO_CORD = os.getenv("PATH_TO_CORD")

In [9]:
# #delete tables
# con = sqlite3.connect('../cord.db')
# cur = con.cursor()

# # Create table
# cur.execute("DROP TABLE relevant_factors")

# # Insert a row of data
# cur.execute("DROP TABLE symptoms")

# # Save (commit) the changes
# con.commit()

# # We can also close the connection if we are done with it.
# # Just be sure any changes have been committed or they will be lost.
# con.close()

In [10]:
def prepare_sliced_df_for_metadata(df, start, end) :
    df_slice = df[start:end]
    df_slice = df_slice.dropna()
    
    #get pdf_json_file location
    pdf_json_file = df_slice.iloc[:, 9]
    
    text = []
    for i, location in enumerate(pdf_json_file) :
        
        path = PATH_TO_CORD + '/' + str(location)
        
        if os.path.isfile(path) :

            f = open(path)

            # returns JSON object as a dictionary
            data = json.load(f)

            text_temp = ''

            count_text = 0
            for body in data['body_text'] :
                text_temp += body['text']
                count_text += 1
                if count_text > 3 :
                    break

            text.append(text_temp)

            # Closing file
            f.close()
            
        else : text.append('')
            
    df_slice['body_text'] = text
    return df_slice

In [11]:
def make_table(path_to_file, to_remove, table_name) :
    df = pd.read_csv(path_to_file)

    df.drop(to_remove,  axis=1, inplace=True, errors='ignore')
    
    df_slice = prepare_sliced_df_for_metadata(df, 0, 5000)
    
    conn = sqlite3.connect('../cord.db')
    df_slice.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    
    return df_slice

In [12]:
def append_to_table(path_to_file, to_remove, table_name) :
    df = pd.read_csv(path_to_file)

    df.drop(to_remove, axis=1, inplace=True, errors='ignore')
    df.drop_duplicates(subset=['Study'], inplace=True)
    conn = sqlite3.connect('../cord.db')
    df.to_sql(table_name, conn, if_exists='append', index=False)
    conn.close()
    
    return df

In [13]:
PATH_TO_CORD = '../data'

In [14]:
to_remove = ['sha', 'source_x', 'pmcid', 'mag_id', 'who_covidence_id', 'arxiv_id', 'pmc_json_files', 's2_id']
table_name = 'cord19'
path_to_file = PATH_TO_CORD + '/metadata.csv'
df = make_table(path_to_file, to_remove, table_name)

/tmp/ipykernel_624923/4253300472.py:4: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  df = make_table(path_to_file, to_remove, table_name)


In [15]:
# to_remove = ['Date', 'Study Type', 'Influential', 'Infuential', 'Influential (Y/N)', 'Date Published', 'Factors Described']
# path_to_folder = '/Kaggle/target_tables/2_relevant_factors/'
# for file_name in os.listdir(PATH_TO_CORD + path_to_folder) :
#     path_to_file = PATH_TO_CORD + path_to_folder + file_name
    
#     table_name = 'relevant_factors'
#     df = append_to_table(path_to_file, to_remove, table_name)

In [16]:
# to_remove = ['Range (Days)', 'Days', 'Date', 'Study Type', 'Aymptomatic', 'Manifestation', 'Frequency of Symptoms', 'Sample Size', 'Specific Sampled Viral load correlated to postive test', 'Age', 'Asymptomatic', 'Sample Obtained', 'Sample obtained', 'Asymptomatic Transmission', 'Characteristic Related to Question 2']
# path_to_folder = '/Kaggle/target_tables/3_patient_descriptions/'
# for file_name in os.listdir(PATH_TO_CORD + path_to_folder) :
#     path_to_file = PATH_TO_CORD + path_to_folder + file_name
    
#     table_name = 'symptoms'
#     df = append_to_table(path_to_file, to_remove, table_name)

In [17]:
conn = sqlite3.connect('../cord.db')
pd.read_sql('SELECT * FROM cord19 LIMIT 0,30', conn)

,cord_uid,title,doi,pubmed_id,license,abstract,publish_time,authors,journal,pdf_json_files,url,body_text
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,document_parses/pdf_json/d1aafb70c066a2068b027...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,document_parses/pdf_json/6b0567729c2143a66d737...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,10.1186/rr19,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,document_parses/pdf_json/06ced00a5fc04215949aa...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,
3,2b73a28n,Role of endothelin-1 in lung disease,10.1186/rr44,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,document_parses/pdf_json/348055649b6b8cf2b9a37...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,
4,9785vg6d,Gene expression in epithelial cells in respons...,10.1186/rr61,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,document_parses/pdf_json/5f48792a5fa08bed9f560...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,
5,zjufx4fo,Sequence requirements for RNA strand transfer ...,10.1093/emboj/20.24.7220,11742998,green-oa,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",The EMBO Journal,document_parses/pdf_json/b2897e1277f56641193a6...,http://europepmc.org/articles/pmc125340?pdf=re...,
6,5yhe786e,Debate: Transfusing to normal haemoglobin leve...,10.1186/cc987,11299062,no-cc,Recent evidence suggests that critically ill p...,2001-03-08,"Alvarez, Gonzalo; Hébert, Paul C; Szick, Sharyn",Crit Care,document_parses/pdf_json/3bb07ea10432f7738413d...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,
7,8zchiykl,The 21st International Symposium on Intensive ...,10.1186/cc1013,11353930,no-cc,The 21st International Symposium on Intensive ...,2001-05-02,"Ball, Jonathan; Venn, Richard",Crit Care,document_parses/pdf_json/5806726a24dc91de39540...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,
8,8qnrcgnk,Heme oxygenase-1 and carbon monoxide in pulmon...,10.1186/1465-9921-4-7,12964953,no-cc,"Heme oxygenase-1 (HO-1), an inducible stress p...",2003-08-07,"Slebos, Dirk-Jan; Ryter, Stefan W; Choi, Augus...",Respir Res,document_parses/pdf_json/faaf1022ccfe93b032c56...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,
9,jg13scgo,Technical Description of RODS: A Real-time Pub...,10.1197/jamia.m1345,12807803,bronze-oa,This report describes the design and implement...,2003-09-01,"Tsui, Fu-Chiang; Espino, Jeremy U.; Dato, Virg...",Journal of the American Medical Informatics As...,document_parses/pdf_json/5b44feca5d6ffaaeb6650...,https://academic.oup.com/jamia/article-pdf/10/...,


In [18]:
conn.close()